# Day 10: Pipe Maze

## Part One

You use the hang glider to ride the hot air from Desert Island all the way up to the floating metal island. This island is surprisingly cold and there definitely aren't any thermals to glide on, so you leave your hang glider behind.

You wander around for a while, but you don't find any people or animals. However, you do occasionally find signposts labeled "Hot Springs" pointing in a seemingly consistent direction; maybe you can find someone at the hot springs and ask them where the desert-machine parts are made.

The landscape here is alien; even the flowers and trees are made of metal. As you stop to admire some metal grass, you notice something metallic scurry away in your peripheral vision and jump into a big pipe! It didn't look like any animal you've ever seen; if you want a better look, you'll need to get ahead of it.

Scanning the area, you discover that the entire field you're standing on is densely packed with pipes; it was hard to tell at first because they're the same metallic silver color as the "ground". You make a quick sketch of all of the surface pipes you can see (your puzzle input).

The pipes are arranged in a two-dimensional grid of tiles:

- `|` is a vertical pipe connecting north and south.
- `-` is a horizontal pipe connecting east and west.
- `L` is a 90-degree bend connecting north and east.
- `J` is a 90-degree bend connecting north and west.
- `7` is a 90-degree bend connecting south and west.
- `F` is a 90-degree bend connecting south and east.
- `.` is ground; there is no pipe in this tile.
- `S` is the starting position of the animal; there is a pipe on this tile, but your sketch doesn't show what shape the pipe has.

Based on the acoustics of the animal's scurrying, you're confident the pipe that contains the animal is one large, continuous loop.

For example, here is a square loop of pipe:

```
.....
.F-7.
.|.|.
.L-J.
.....
```

If the animal had entered this loop in the northwest corner, the sketch would instead look like this:

```
.....
.S-7.
.|.|.
.L-J.
.....
```

In the above diagram, the S tile is still a 90-degree F bend: you can tell because of how the adjacent pipes connect to it.

Unfortunately, there are also many pipes that aren't connected to the loop! This sketch shows the same loop as above:

In [28]:
example1 = [
    "-L|F7",
    "7S-7|",
    "L|7||",
    "-L-J|",
    "L|-JF"
]

5-element Vector{String}:
 "-L|F7"
 "7S-7|"
 "L|7||"
 "-L-J|"
 "L|-JF"

In the above diagram, you can still figure out which pipes form the main loop: they're the ones connected to S, pipes those pipes connect to, pipes those pipes connect to, and so on. Every pipe in the main loop connects to its two neighbors (including S, which will have exactly two pipes connecting to it, and which is assumed to connect back to those two pipes).

Here is a sketch that contains a slightly more complex main loop:

```
..F7.
.FJ|.
SJ.L7
|F--J
LJ...
```

Here's the same example sketch with the extra, non-main-loop pipe tiles also shown:

In [29]:
example2 = [
    "7-F7-",
    ".FJ|7",
    "SJLL7",
    "|F--J",
    "LJ.LJ"
]

5-element Vector{String}:
 "7-F7-"
 ".FJ|7"
 "SJLL7"
 "|F--J"
 "LJ.LJ"

If you want to get out ahead of the animal, you should find the tile in the loop that is farthest from the starting position. Because the animal is in the pipe, it doesn't make sense to measure this by direct distance. Instead, you need to find the tile that would take the longest number of steps along the loop to reach from the starting point - regardless of which way around the loop the animal went.

In the first example with the square loop:

```
.....
.S-7.
.|.|.
.L-J.
.....
```
You can count the distance each tile in the loop is from the starting point like this:

```
.....
.012.
.1.3.
.234.
.....
```
In this example, the farthest point from the start is 4 steps away.

Here's the more complex loop again:

```
..F7.
.FJ|.
SJ.L7
|F--J
LJ...
```
Here are the distances for each tile on that loop:

```
..45.
.236.
01.78
14567
23...
```

Find the single giant loop starting at S. How many steps along the loop does it take to get from the starting position to the point farthest from the starting position?

In [30]:
using ResumableFunctions

function ispossible(pipe, direction)
    if direction == (1, 0)
        return pipe == '|' || pipe == 'L' || pipe == 'J'
    elseif direction == (-1, 0)
        return pipe == '|' || pipe == '7' || pipe == 'F'
    elseif direction == (0, 1)
        return pipe == '-' || pipe == '7' || pipe == 'J'
    elseif direction == (0, -1)
        return pipe == '-' || pipe == 'L' || pipe == 'F'
    end
    return false
end

function isinbounds(contents, position)
    if position[1] < firstindex(contents) || position[1] > lastindex(contents)
        return false
    end
    if position[2] < firstindex(contents[position[1]]) || position[2] > lastindex(contents[position[1]])
        return false
    end
    return true
end

function nextdirection(pipe, direction)
    if pipe == 'L'
        direction .+ (-1, 1)
    elseif pipe == 'J'
        direction .+ (-1, -1)
    elseif pipe == '7'
        direction .+ (1, -1)
    elseif pipe == 'F'
        direction .+ (1, 1)
    else
        direction
    end
end

@resumable function walk(contents, initialposition, initialdirection)
    d = initialdirection
    p = initialposition
    s = contents[p[1]][p[2]]
    while ispossible(s, d)
        @yield p
        d = nextdirection(s, d)
        p = p .+ d
        if !isinbounds(contents, p)
            break
        end
        s = contents[p[1]][p[2]]
    end
end

function findstart(contents)
    for y in firstindex(contents):lastindex(contents)
        for x in firstindex(contents[y]):lastindex(contents[y])
            if contents[y][x] == 'S'
                return (y, x)
            end
        end
    end
end

findstart (generic function with 1 method)

In [31]:
function partone(contents)
    startposition = findstart(contents)
    for (step, coords...) in zip(Iterators.countfrom(1), map(d -> walk(contents, startposition .+ d, d),
        filter(
            function (direction)
                p = startposition .+ direction
                return isinbounds(contents, p) && ispossible(contents[p[1]][p[2]], direction)
            end,
            [(0, 1), (0, -1), (1, 0), (-1, 0)]
        ))...)
        if coords[1] == coords[2]
            return step
        end
    end
end

println(partone(example1))
println(partone(example2))

4
8


## Part Two

You quickly reach the farthest point of the loop, but the animal never emerges. Maybe its nest is within the area enclosed by the loop?

To determine whether it's even worth taking the time to search for such a nest, you should calculate how many tiles are contained within the loop. For example:

In [32]:
example3 = [
    "...........",
    ".S-------7.",
    ".|F-----7|.",
    ".||.....||.",
    ".||.....||.",
    ".|L-7.F-J|.",
    ".|..|.|..|.",
    ".L--J.L--J.",
    "..........."
]

9-element Vector{String}:
 "..........."
 ".S-------7."
 ".|F-----7|."
 ".||.....||."
 ".||.....||."
 ".|L-7.F-J|."
 ".|..|.|..|."
 ".L--J.L--J."
 "..........."

The above loop encloses merely four tiles - the two pairs of . in the southwest and southeast (marked I below). The middle . tiles (marked O below) are not in the loop. Here is the same loop again with those regions marked:

```
...........
.S-------7.
.|F-----7|.
.||OOOOO||.
.||OOOOO||.
.|L-7OF-J|.
.|II|O|II|.
.L--JOL--J.
.....O.....
```

In fact, there doesn't even need to be a full tile path to the outside for tiles to count as outside the loop - squeezing between pipes is also allowed! Here, I is still within the loop and O is still outside the loop:

```
..........
.S------7.
.|F----7|.
.||OOOO||.
.||OOOO||.
.|L-7F-J|.
.|II||II|.
.L--JL--J.
..........
```

In both of the above examples, 4 tiles are enclosed by the loop.

Here's a larger example:

```
.F----7F7F7F7F-7....
.|F--7||||||||FJ....
.||.FJ||||||||L7....
FJL7L7LJLJ||LJ.L-7..
L--J.L7...LJS7F-7L7.
....F-J..F7FJ|L7L7L7
....L7.F7||L7|.L7L7|
.....|FJLJ|FJ|F7|.LJ
....FJL-7.||.||||...
....L---J.LJ.LJLJ...
```

The above sketch has many random bits of ground, some of which are in the loop (I) and some of which are outside it (O):

```
OF----7F7F7F7F-7OOOO
O|F--7||||||||FJOOOO
O||OFJ||||||||L7OOOO
FJL7L7LJLJ||LJIL-7OO
L--JOL7IIILJS7F-7L7O
OOOOF-JIIF7FJ|L7L7L7
OOOOL7IF7||L7|IL7L7|
OOOOO|FJLJ|FJ|F7|OLJ
OOOOFJL-7O||O||||OOO
OOOOL---JOLJOLJLJOOO
```

In this larger example, 8 tiles are enclosed by the loop.

Any tile that isn't part of the main loop can count as being enclosed by the loop. Here's another example with many bits of junk pipe lying around that aren't connected to the main loop at all:

In [33]:
example4 = [
    "FF7FSF7F7F7F7F7F---7",
    "L|LJ||||||||||||F--J",
    "FL-7LJLJ||||||LJL-77",
    "F--JF--7||LJLJ7F7FJ-",
    "L---JF-JLJ.||-FJLJJ7",
    "|F|F-JF---7F7-L7L|7|",
    "|FFJF7L7F-JF7|JL---7",
    "7-L-JL7||F7|L7F-7F7|",
    "L.L7LFJ|||||FJL7||LJ",
    "L7JLJL-JLJLJL--JLJ.L"
]

10-element Vector{String}:
 "FF7FSF7F7F7F7F7F---7"
 "L|LJ||||||||||||F--J"
 "FL-7LJLJ||||||LJL-77"
 "F--JF--7||LJLJ7F7FJ-"
 "L---JF-JLJ.||-FJLJJ7"
 "|F|F-JF---7F7-L7L|7|"
 "|FFJF7L7F-JF7|JL---7"
 "7-L-JL7||F7|L7F-7F7|"
 "L.L7LFJ|||||FJL7||LJ"
 "L7JLJL-JLJLJL--JLJ.L"

Here are just the tiles that are enclosed by the loop marked with I:

```
FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJIF7FJ-
L---JF-JLJIIIIFJLJJ7
|F|F-JF---7IIIL7L|7|
|FFJF7L7F-JF7IIL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L
```

In this last example, 10 tiles are enclosed by the loop.

Figure out whether you have time to search for the nest by calculating the area within the loop. How many tiles are enclosed by the loop?

In [34]:
## a set of coordinates to track which are the ones in the loop, and which are extra pipe that we can ignore.
function loopindexes(contents)
    startposition = findstart(contents)
    indexes = Set([startposition])
    for d in [(0, 1), (0, -1), (1, 0), (-1, 0)]
        p = startposition .+ d
        if isinbounds(contents, p) && ispossible(contents[p[1]][p[2]], d)
            for n in walk(contents, p, d)
                push!(indexes, n)
                if n === startposition
                    break
                end
            end
            break
        end
    end
    return indexes
end

## figure out what pipe should be used in place of S basically solve for the pipe that fits into the loop in it's place.
function determines(contents, sposition)
    (top, right, bottom, left) = map(
        function (d)
            p = sposition .+ d
            return isinbounds(contents, p) && ispossible(contents[p[1]][p[2]], d)
        end,
        [(-1, 0), (0, 1), (1, 0), (0, -1)]
    )
    if top
        if right
            return 'L'
        elseif bottom
            return '|'
        else
            return 'J'
        end
    elseif bottom
        if right
            return 'F'
        else
            return '7'
        end
    else
        return '-'
    end
end

## adaptation of the even/odd rule for shape filing in vector graphics
function isenclosed(contents, position, indexes)
    ## go from the position to the left edge in a stright line
    ## track the number of times we cross a | pipe
    intersections = 0
    ## track how many corners we could go over (F, 7)
    unders = 0
    ## track how many corners we could go under (L, J)
    overs = 0
    for x in 1:position[2]
        if (position[1], x) in indexes
            s = contents[position[1]][x]
            if s == 'S'
                s = determines(contents, (position[1], x))
            end
            if s == '|'
                intersections += 1
            elseif s == '7' || s == 'F'
                overs += 1
            elseif s == 'L' || s == 'J'
                unders += 1
            end
        end
    end
    ## we must cross | pipes, we have to cross a pair of corers that we can go over and under, use min to count pairs
    ## eg. |F7J <- this is 2 crossings since we go through the |, over the F and 7, and through the J
    ## if we cross an odd number of lines then we must be encosed by the shape, otherwise we are not.
    return isodd(intersections + min(unders, overs))
end

isenclosed (generic function with 1 method)

In [35]:
function parttwo(contents)
    indexes = loopindexes(contents)
    enclosed = 0
    for y in firstindex(contents):lastindex(contents)
        for x in firstindex(contents[y]):lastindex(contents[y])
            if (y, x) in indexes
                continue
            elseif isenclosed(contents, (y, x), indexes)
                enclosed += 1
            end
        end
    end
    return enclosed
end

println(parttwo(example3))
println(parttwo(example4))

4
10


## Results

In [36]:
include("./aoc.jl")

execute(10, partone, parttwo)

7063
589
